In [1]:
import pandas as pd
import numpy as np

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [3]:
csv_file_1 = "../ETL_Project_Corona/2019_nCoV_data.csv"
corona_data_df = pd.read_csv(csv_file_1)
corona_data_df.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,1/22/2020 12:00,Anhui,China,1/22/2020 12:00,1,0,0
1,2,1/22/2020 12:00,Beijing,China,1/22/2020 12:00,14,0,0
2,3,1/22/2020 12:00,Chongqing,China,1/22/2020 12:00,6,0,0
3,4,1/22/2020 12:00,Fujian,China,1/22/2020 12:00,1,0,0
4,5,1/22/2020 12:00,Gansu,China,1/22/2020 12:00,0,0,0


In [4]:
# corona_data_df = corona_data_df.rename(columns={"Province/State": "Province_State"})
# corona_data_df

In [5]:
countries =  corona_data_df['Country'].unique()
countries

array(['China', 'US', 'Japan', 'Thailand', 'South Korea',
       'Mainland China', 'Hong Kong', 'Macau', 'Taiwan', 'Singapore',
       'Philippines', 'Malaysia', 'Vietnam', 'Australia', 'Mexico',
       'Brazil', 'France', 'Nepal', 'Canada', 'Cambodia', 'Sri Lanka',
       'Ivory Coast', 'Germany', 'Finland', 'United Arab Emirates',
       'India', 'Italy', 'Sweden', 'Russia', 'Spain', 'UK', 'Belgium',
       'Others'], dtype=object)

In [6]:
Province_State = corona_data_df['Province/State'].unique()
Province_State

array(['Anhui', 'Beijing', 'Chongqing', 'Fujian', 'Gansu', 'Guangdong',
       'Guangxi', 'Guizhou', 'Hainan', 'Hebei', 'Heilongjiang', 'Henan',
       'Hong Kong', 'Hubei', 'Hunan', 'Inner Mongolia', 'Jiangsu',
       'Jiangxi', 'Jilin', 'Liaoning', 'Macau', 'Ningxia', 'Qinghai',
       'Shaanxi', 'Shandong', 'Shanghai', 'Shanxi', 'Sichuan', 'Taiwan',
       'Tianjin', 'Tibet', 'Washington', 'Xinjiang', 'Yunnan', 'Zhejiang',
       nan, 'Chicago', 'Illinois', 'California', 'Arizona', 'Ontario',
       'New South Wales', 'Victoria', 'Bavaria', 'British Columbia',
       'Queensland', 'Chicago, IL', 'South Australia', 'Boston, MA',
       'Los Angeles, CA', 'Orange, CA', 'Santa Clara, CA', 'Seattle, WA',
       'Tempe, AZ', 'Toronto, ON', 'San Benito, CA', 'London, ON',
       'Madison, WI', 'Cruise Ship', 'Diamond Princess cruise ship'],
      dtype=object)

In [7]:
country_df= corona_data_df.groupby([ 'Country'])
country_df.head()

,Sno,Date,Province/State,Country,Last Update,Confirmed,Deaths,Recovered
0,1,1/22/2020 12:00,Anhui,China,1/22/2020 12:00,1,0,0
1,2,1/22/2020 12:00,Beijing,China,1/22/2020 12:00,14,0,0
2,3,1/22/2020 12:00,Chongqing,China,1/22/2020 12:00,6,0,0
3,4,1/22/2020 12:00,Fujian,China,1/22/2020 12:00,1,0,0
4,5,1/22/2020 12:00,Gansu,China,1/22/2020 12:00,0,0,0
...,...,...,...,...,...,...,...,...
968,969,2/7/2020 20:24,NaN,Belgium,4/2/2020 15:43,1,0,0
1009,1010,2/8/2020 23:04,Cruise Ship,Others,7/2/2020 16:33,61,0,0
1040,1041,2/8/2020 23:04,NaN,Belgium,4/2/2020 15:43,1,0,0
1081,1082,2/9/2020 23:20,Diamond Princess cruise ship,Others,2/9/2020 6:13,64,0,0


In [8]:
#Confirmed to death to recovery ratio, you can deduced the results are based the availibility/ quality of medical facilities. 
#Mainland china does not have enough facilities and personel to treat all confirmed cases. Higher death rates result.
total_cdr_country_df = country_df["Confirmed","Deaths","Recovered"].sum()
total_cdr_country_df.head(37)

,Confirmed,Deaths,Recovered
Country,,,
Australia,179,0,24
Belgium,7,0,0
Brazil,0,0,0
Cambodia,15,0,0
Canada,66,0,0
China,549,0,0
Finland,13,0,0
France,106,0,0
Germany,140,0,0


In [49]:
total_cdr_country_df.insert(0, 'Id', range(1, 1 + len(total_cdr_country_df)))

In [50]:
total_cdr_country_df = total_cdr_country_df.set_index("Id")
total_cdr_country_df.head()

,Confirmed,Deaths,Recovered
Id,,,
1,179,0,24
2,7,0,0
3,0,0,0
4,15,0,0
5,66,0,0


In [9]:
c_country_df = country_df["Confirmed"].sum()
c_country_df 

Country
Australia                  179
Belgium                      7
Brazil                       0
Cambodia                    15
Canada                      66
China                      549
Finland                     13
France                     106
Germany                    140
Hong Kong                  300
India                       29
Italy                       28
Ivory Coast                  0
Japan                      312
Macau                      139
Mainland China          327215
Malaysia                   163
Mexico                       0
Nepal                       17
Others                     321
Philippines                 26
Russia                      22
Singapore                  351
South Korea                245
Spain                       13
Sri Lanka                   15
Sweden                      11
Taiwan                     188
Thailand                   347
UK                          31
US                         149
United Arab Emirates        68


In [10]:
d_country_df = country_df["Deaths"].sum()
d_country_df 

Country
Australia                  0
Belgium                    0
Brazil                     0
Cambodia                   0
Canada                     0
China                      0
Finland                    0
France                     0
Germany                    0
Hong Kong                  7
India                      0
Italy                      0
Ivory Coast                0
Japan                      0
Macau                      0
Mainland China          7131
Malaysia                   0
Mexico                     0
Nepal                      0
Others                     0
Philippines               10
Russia                     0
Singapore                  0
South Korea                0
Spain                      0
Sri Lanka                  0
Sweden                     0
Taiwan                     0
Thailand                   0
UK                         0
US                         0
United Arab Emirates       0
Vietnam                    0
Name: Deaths, dtype: int64

In [11]:
r_country_df = country_df["Recovered"].sum()
r_country_df 

Country
Australia                  24
Belgium                     0
Brazil                      0
Cambodia                    0
Canada                      0
China                       0
Finland                     0
France                      0
Germany                     0
Hong Kong                   0
India                       0
Italy                       0
Ivory Coast                 0
Japan                      19
Macau                       5
Mainland China          17343
Malaysia                    4
Mexico                      0
Nepal                       0
Others                      0
Philippines                 0
Russia                      0
Singapore                   6
South Korea                10
Spain                       0
Sri Lanka                   3
Sweden                      0
Taiwan                      5
Thailand                   91
UK                          0
US                          6
United Arab Emirates        0
Vietnam                    10
Na

In [12]:
recovery_percentages= r_country_df.value_counts()/c_country_df.count()*100

recovery_percentages

0        63.636364
10        6.060606
6         6.060606
5         6.060606
17343     3.030303
91        3.030303
24        3.030303
19        3.030303
4         3.030303
3         3.030303
Name: Recovered, dtype: float64

In [13]:
recovery_percentges_country_df= pd.DataFrame({
                                          
                                          "Recovered Percentage": [recovery_percentages]
                                        
})
recovery_percentges_country_df

,Recovered Percentage
0,0 63.636364 10 6.060606 6 ...


In [14]:
new_corona_data_df = corona_data_df[['Sno','Last Update', 'Country','Province/State','Confirmed','Deaths','Recovered']].copy()
new_corona_data_df

,Sno,Last Update,Country,Province/State,Confirmed,Deaths,Recovered
0,1,1/22/2020 12:00,China,Anhui,1,0,0
1,2,1/22/2020 12:00,China,Beijing,14,0,0
2,3,1/22/2020 12:00,China,Chongqing,6,0,0
3,4,1/22/2020 12:00,China,Fujian,1,0,0
4,5,1/22/2020 12:00,China,Gansu,0,0,0
...,...,...,...,...,...,...,...
1194,1195,2/1/2020 19:53,US,"Los Angeles, CA",1,0,0
1195,1196,2/5/2020 21:53,US,"Madison, WI",1,0,0
1196,1197,2/1/2020 19:53,US,"Orange, CA",1,0,0
1197,1198,2/9/2020 7:03,US,"Seattle, WA",1,0,1


In [15]:
new_corona_data_df = new_corona_data_df.rename(columns={"Sno": "Id"})

In [16]:
# new_corona_data_df = new_corona_data_df.rename(columns={"Province/State": "Province_State"})

In [17]:
# new_corona_data_df.set_index("Id")
new_corona_data_df = new_corona_data_df.set_index("Id")
new_corona_data_df.head()

,Last Update,Country,Province/State,Confirmed,Deaths,Recovered
Id,,,,,,
1,1/22/2020 12:00,China,Anhui,1,0,0
2,1/22/2020 12:00,China,Beijing,14,0,0
3,1/22/2020 12:00,China,Chongqing,6,0,0
4,1/22/2020 12:00,China,Fujian,1,0,0
5,1/22/2020 12:00,China,Gansu,0,0,0


In [18]:
# date_df= new_corona_data_df.groupby([ 'Last Update'])
# date_df.head()

In [19]:
csv_file_2 = "../ETL_Project_Corona/time_series_2019_ncov_confirmed.csv"
confirmed_data_df = pd.read_csv(csv_file_2)
confirmed_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 22:04,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,39.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,30.05718,107.8740,5.0,6.0,9.0,27.0,27.0,57.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,26.07783,117.9895,NaN,1.0,5.0,5.0,10.0,10.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,2.0,2.0,2.0,4.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83


In [20]:
confirmed_data_df = confirmed_data_df.drop(['Lat','Long'], axis=1)

In [21]:
confirmed_data_df.insert(0, 'Id', range(1, 1 + len(confirmed_data_df)))

In [22]:
confirmed_data_df = confirmed_data_df.set_index("Id")

In [23]:
# confirmed_data_df = confirmed_data_df.rename(columns={"Country/Region": "Country"})

In [24]:
# confirmed_data_df = confirmed_data_df.rename(columns={"Province/State": "Province_State"})
confirmed_data_df

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 22:04,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
Id,,,,,,,,,,,,,,,,,,,,,
1,Anhui,Mainland China,NaN,1.0,9.0,15.0,15.0,39.0,39.0,60.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
2,Beijing,Mainland China,10.0,14.0,22.0,26.0,36.0,36.0,41.0,51.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
3,Chongqing,Mainland China,5.0,6.0,9.0,27.0,27.0,57.0,57.0,75.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
4,Fujian,Mainland China,NaN,1.0,5.0,5.0,10.0,10.0,18.0,18.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
5,Gansu,Mainland China,NaN,NaN,2.0,2.0,2.0,4.0,4.0,7.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
69,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
70,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1


In [25]:
csv_file_3 = "../ETL_Project_Corona/time_series_2019_ncov_deaths.csv"
deaths_data_df = pd.read_csv(csv_file_3)
deaths_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
1,Beijing,Mainland China,40.18238,116.4142,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
2,Chongqing,Mainland China,30.05718,107.8740,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
3,Fujian,Mainland China,26.07783,117.9895,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2


In [26]:
deaths_data_df = deaths_data_df.drop(['Lat','Long'], axis=1)

In [27]:
deaths_data_df.insert(0, 'Id', range(1, 1 + len(deaths_data_df)))

In [28]:
deaths_data_df = deaths_data_df.set_index("Id")

In [29]:
# deaths_data_df = deaths_data_df.rename(columns={"Country/Region": "Country"})

In [30]:
# deaths_data_df = deaths_data_df.rename(columns={"Province/State": "Province_State"})
deaths_data_df

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
Id,,,,,,,,,,,,,,,,,,,,,
1,Anhui,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
2,Beijing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
3,Chongqing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
4,Fujian,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
5,Gansu,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
69,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
70,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [31]:
csv_file_4 = "../ETL_Project_Corona/time_series_2019_ncov_deaths.csv"
recovered_data_df = pd.read_csv(csv_file_4)
recovered_data_df.head()

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
1,Beijing,Mainland China,40.18238,116.4142,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
2,Chongqing,Mainland China,30.05718,107.8740,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
3,Fujian,Mainland China,26.07783,117.9895,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2


In [32]:
recovered_data_df = recovered_data_df.drop(['Lat','Long'], axis=1)

In [33]:
recovered_data_df.insert(0, 'Id', range(1, 1 + len(recovered_data_df)))

In [34]:
recovered_data_df = recovered_data_df.set_index("Id")

In [35]:
# recovered_data_df = recovered_data_df.rename(columns={"Country/Region": "Country"})

In [36]:
# recovered_data_df = recovered_data_df.rename(columns={"Province/State": "Province_State"})
recovered_data_df

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 10:24,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
Id,,,,,,,,,,,,,,,,,,,,,
1,Anhui,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,1,3
2,Beijing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,2.0,2,2,2,2
3,Chongqing,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2.0,2.0,2,2,2,2
4,Fujian,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
5,Gansu,Mainland China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
69,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
70,NaN,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [37]:
rds_connection_string = "postgres:postgres@localhost:5432/Corona_db"
engine = create_engine(f'postgresql://{rds_connection_string}')


In [38]:
total_cdr_country_df.to_sql(name='total_cdr_country', con=engine,  if_exists='append', index=False)

In [39]:
new_corona_data_df.to_sql(name='corona_data', con=engine, if_exists='append', index=False)

In [40]:
confirmed_data_df.to_sql(name='confirmed_data', con=engine, if_exists='append', index=False)

In [41]:
deaths_data_df .to_sql(name='deaths_data', con=engine, if_exists='append', index=False)

In [42]:
recovered_data_df .to_sql(name='recovered_data', con=engine, if_exists='append', index=False)

In [43]:
pd.read_sql_query('select * from total_cdr_country', con=engine)

,Confirmed,Deaths,Recovered
0,179,0,24
1,7,0,0
2,0,0,0
3,15,0,0
4,66,0,0
...,...,...,...
523,347,0,91
524,31,0,0
525,149,0,6
526,68,0,0


In [44]:
pd.read_sql_query('select * from corona_data', con=engine)

,Last Update,Country,Province/State,Confirmed,Deaths,Recovered
0,1/22/2020 12:00,China,Anhui,1,0,0
1,1/22/2020 12:00,China,Beijing,14,0,0
2,1/22/2020 12:00,China,Chongqing,6,0,0
3,1/22/2020 12:00,China,Fujian,1,0,0
4,1/22/2020 12:00,China,Gansu,0,0,0
...,...,...,...,...,...,...
15582,2/1/2020 19:53,US,"Los Angeles, CA",1,0,0
15583,2/5/2020 21:53,US,"Madison, WI",1,0,0
15584,2/1/2020 19:53,US,"Orange, CA",1,0,0
15585,2/9/2020 7:03,US,"Seattle, WA",1,0,1


In [45]:
pd.read_sql_query('select * from confirmed_data', con=engine)

,Province/State,Country/Region,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,...,02/05/20 9:00,02/05/20 23:00,02/06/20 9:00,02/06/20 14:20,02/07/20 20:13,02/07/20 22:50,02/08/20 22:04,02/08/20 23:04,02/09/20 10:30,02/09/20 23:20
0,Anhui,Mainland China,NaN,1.0,9.0,15.0,15.0,39.0,39.0,60.0,...,530.0,591.0,591.0,591.0,665,733,733,779,779,830
1,Beijing,Mainland China,10.0,14.0,22.0,26.0,36.0,36.0,41.0,51.0,...,253.0,274.0,274.0,274.0,297,315,315,326,326,337
2,Chongqing,Mainland China,5.0,6.0,9.0,27.0,27.0,57.0,57.0,75.0,...,376.0,389.0,400.0,400.0,415,426,428,446,450,468
3,Fujian,Mainland China,NaN,1.0,5.0,5.0,10.0,10.0,18.0,18.0,...,205.0,215.0,215.0,215.0,224,239,239,250,250,261
4,Gansu,Mainland China,NaN,NaN,2.0,2.0,2.0,4.0,4.0,7.0,...,57.0,62.0,62.0,62.0,67,71,79,79,79,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,"Boston, MA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
932,"San Benito, CA",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,2.0,2.0,2.0,2,2,2,2,2,2
933,None,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,1,1,1,1,1
934,"Madison, WI",US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,1.0,1,1,1,1,1,1


In [46]:
engine.table_names()

['total_cdr_country',
 'corona_data',
 'confirmed_data',
 'deaths_data',
 'recovered_data']